# Ensemble 202108

#### 2.1  Seteo del ambiente en Google Colab

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_02_crudo.csv.gz"
archivo_destino="/content/datasets/competencia_02_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_02_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


## Inicializacion

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-16 14:18:00 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658289,35.2,1454465,77.7,1388163,74.2
Vcells,1222640,9.4,8388608,64.0,1975128,15.1


In [ ]:
PARAM <- list()
PARAM$experimento <- "zensemble-202108"
PARAM$semilla_primigenia <- 202513

In [ ]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### Ensemble

In [ ]:
library(data.table)

In [ ]:
PARAM$foto_mes_filtro <- 202108

In [ ]:
modelos_info <- data.table(

  experimento_id = c("seg-001",
                     "zmuerte-10-2",
                     "zmuerte-50-2",
                     "apo-006"),

  score = c(399999,
            385599,
            388799,
            399759),

  archivo_pred = c("prediccion.txt",
                   "prediccion.txt",
                   "prediccion.txt",
                   "prediccion.txt")
)

RUTA_BASE_EXP <- "/content/buckets/b1/exp/"

# Weights

score_total <- sum(modelos_info$score)
modelos_info[, peso := score / score_total]

# Ensamble

tb_ensamble <- NULL


for (i in 1:nrow(modelos_info)) {

  modelo_actual <- modelos_info[i]

  # Construyo la ruta completa al archivo de predicción
  ruta_completa <- paste0(
    RUTA_BASE_EXP,
    modelo_actual$experimento_id, "/",
    modelo_actual$archivo_pred
  )

  tryCatch({
    # Leo el archivo de predicción
    tb_pred <- fread(ruta_completa)

    # Verifico que tenga las columnas esperadas
    if (!all(c("numero_de_cliente", "prob") %in% names(tb_pred))) {
      next
    }

    # Renombro la columna 'prob' para que sea única de este modelo
    col_prob_nueva <- paste0("prob_", modelo_actual$experimento_id)
    setnames(tb_pred, "prob", col_prob_nueva)

    tb_pred <- tb_pred[, c("numero_de_cliente", col_prob_nueva), with = FALSE]

    # Ensamble
    if (is.null(tb_ensamble)) {
      tb_ensamble <- tb_pred
    } else {
      tb_ensamble <- merge(tb_ensamble, tb_pred, by = "numero_de_cliente", all = TRUE)
    }

  }, error = function(e)
  )
}

# Ensamble Ponderado

if (!is.null(tb_ensamble)) {

  tb_ensamble[, prob_ensamble := 0.0]

  for (i in 1:nrow(modelos_info)) {

    modelo_actual <- modelos_info[i]
    col_prob <- paste0("prob_", modelo_actual$experimento_id)
    peso_actual <- modelo_actual$peso

    if (col_prob %in% names(tb_ensamble)) {

      tb_ensamble[, prob_ensamble := prob_ensamble + (get(col_prob) * peso_actual), ]
    }
  }
}

  # Me quedo solo con la predicción final
  tb_final <- tb_ensamble[, .(numero_de_cliente, prob = prob_ensamble)]

### Clasificacion

In [ ]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_final, -prob)

envios <- 11000
tb_final[, Predicted := 0L] # seteo inicial a 0
tb_final[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_final[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [ ]:
Sys.time()

[1] "2025-11-16 14:25:44 UTC"